# Importing Libraries

In [17]:
pwd

'/home/wsuser/work'

In [18]:
!pip install keras==2.2.4
!pip install tensorflow==1.14.0

     |████████████████████████████████| 312 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 14.2 MB/s eta 0:00:01
ERROR: Could not find a version that satisfies the requirement tensorflow==1.14.0 (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.7.0rc0, 2.7.0rc1)
ERROR: No matching distribution found for tensorflow==1.14.0


In [19]:
import tensorflow as tf

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

In [21]:
#importing Image datagenerator library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image Data Augmentation

In [22]:
#setting parameter for imge data agumentation to the training data
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.1,
                                 horizontal_flip=True)

In [23]:
#image data augmentation to the testing data
val_datagen=ImageDataGenerator(rescale=1./255)

# Loading our Data And Performing Data Agumentaion

In [24]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_6bba542f401043f5a1f710901ca6cd3b = 'https://s3.eu.cloud-object-storage.appdomain.cloud'
else:
    endpoint_6bba542f401043f5a1f710901ca6cd3b = 'https://s3.private.eu.cloud-object-storage.appdomain.cloud'

client_6bba542f401043f5a1f710901ca6cd3b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='2xIS1tw6ml6iz93wxYjUWtgBRTIdeBiH4A62liAleDGo',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_6bba542f401043f5a1f710901ca6cd3b)

streaming_body_6 = client_6bba542f401043f5a1f710901ca6cd3b.get_object(Bucket='garbageclassification-donotdelete-pr-hnuwvijmaqeoxz', Key='dataset-resized (2).zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [25]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_6.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [26]:
pwd

'/home/wsuser/work'

In [27]:
train_transform=train_datagen.flow_from_directory("/home/wsuser/work/dataset-resized/Training",
                                                target_size=(128,128),
                                                batch_size=64,
                                                class_mode="categorical")

Found 2057 images belonging to 6 classes.


In [28]:
test_transform=val_datagen.flow_from_directory("/home/wsuser//work/dataset-resized/Testing",
                                              target_size=(128,128),
                                              batch_size=64,
                                              class_mode="categorical")

Found 471 images belonging to 6 classes.


In [29]:
#Initializing the model
model=Sequential()

In [30]:
#1)convolution layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [31]:
#1)maxpooling layer
model.add(MaxPooling2D(2,2))

In [32]:
#2)convolution layer
model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))


In [33]:
#2)maxpooling layer
model.add(MaxPooling2D(pool_size=2))

In [34]:
#3)convolution layer and pooling
model.add(Convolution2D(32,(3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

In [35]:
#4)convolution layer and pooling
model.add(Convolution2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

In [36]:
#Flattening the layers
model.add(Flatten())

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        9

In [38]:
#Adding first hidden layer
model.add(Dense(kernel_initializer='uniform',activation='relu',units=150))

In [39]:
#Adding second hidden layer
model.add(Dense(kernel_initializer='uniform',activation='relu',units=68))

In [40]:
#Adding third hidden layer
model.add(Dense(kernel_initializer='uniform',activation='softmax',units=6))

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        9

# Compiling the Model

In [42]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

# Fit the Model

In [43]:
res=model.fit_generator(train_transform,steps_per_epoch=2057//64,validation_steps=471//64,epochs=30,validation_data=test_transform)

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
32/32 [==============================] - 134s 4s/step - loss: 1.7384 - acc: 0.2160 - val_loss: 1.7383 - val_acc: 0.2232
Epoch 2/30
32/32 [==============================] - 132s 4s/step - loss: 1.6930 - acc: 0.2465 - val_loss: 1.7325 - val_acc: 0.2433
Epoch 3/30
32/32 [==============================] - 131s 4s/step - loss: 1.6925 - acc: 0.2646 - val_loss: 1.6709 - val_acc: 0.2924
Epoch 4/30
32/32 [==============================] - 132s 4s/step - loss: 1.5541 - acc: 0.3280 - val_loss: 1.4797 - val_acc: 0.3549
Epoch 5/30
32/32 [==============================] - 131s 4s/step - loss: 1.3913 - acc: 0.4223 - val_loss: 1.4058 - val_acc: 0.3929
Epoch 6/30
32/32 [==============================] - 132s 4s/step - loss: 1.3660 - acc: 0.4213 - val_loss: 1.4179 - val_acc: 0.3862
Epoch 7/30
32/32 [==============================] - 131s 4s/step - loss: 1.2925 - acc: 0.4554 - val_loss: 1.4779 - val_acc: 0.3772
Epoch 8/30
32/32 [==============================] - 130s 4s/step - loss: 1.2694 - a

# Save the Model

In [44]:
model.save('garbage2.h5')

In [45]:
!tar -zcvf Garbage-classification-model_new.tgz garbage2.h5

garbage2.h5


In [46]:
ls -l

total 6740
drwxrwx--- 4 wsuser wscommon    4096 Oct 30 17:16 dataset-resized/
-rw-rw---- 1 wsuser wscommon 3586704 Oct 30 18:23 garbage2.h5
-rw-rw---- 1 wsuser wscommon 3306891 Oct 30 18:23 Garbage-classification-model_new.tgz


In [47]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 15.8 MB/s eta 0:00:01


In [48]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                 "url":"https://eu-gb.ml.cloud.ibm.com",
                 "apikey":"FDzI_6MhRy6ysNq1qo_uMo4IGLYFo2UtY7IMibhivUIM"
                  }
client=APIClient(wml_credentials)

In [49]:
client=APIClient(wml_credentials)

In [50]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [51]:
space_uid=guid_from_space_name(client,'Garbageclassification')
print("Space UID="+space_uid)

Space UID=58dbae38-fecc-4f8f-832a-28d5a67a1181


In [52]:
client.set.default_space(space_uid)

'SUCCESS'

In [53]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [54]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid

'2b73a275-7cbf-420b-a912-eae7f436e0bc'

In [55]:
 model_details=client.repository.store_model(model="Garbage-classification-model_new.tgz",meta_props={
 client.repository.ModelMetaNames.NAME:"CNN",
 client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
 client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}  
                                            )

model_id=client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [56]:
model_id

'061295c7-7086-4388-909b-356ca1e45793'

In [57]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [73]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


In [74]:
model=load_model("garbage2.h5")

In [107]:

streaming_body_7 = client_6bba542f401043f5a1f710901ca6cd3b.get_object(Bucket='garbageclassification-donotdelete-pr-hnuwvijmaqeoxz', Key='plastic419.jpg')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
img=image.load_img('/home/wsuser/work/dataset-resized/Testing/plastic/plastic419.jpg',target_size=(128,128))

In [108]:
import numpy as np
#from tensorflow.keras.models import load_model
#from tensorflow.keras.preprocessing import image
#model=load_model("garbage2.h5")

# Testing

In [109]:


x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

In [110]:
a=np.argmax(model.predict(x),axis=1)

In [111]:
train_transform.class_indices

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [112]:
index=["cardbord","glass","metal","paper","plastic","trash"]
result=str(index[a[0]])
result

'plastic'